# Algoritmos Minimax e Alfabeta

In [ ]:
from collections import namedtuple
import random

from utils import argmax
from canvas import Canvas

infinity = float('inf')
GameState = namedtuple('GameState', 'to_move, utility, board, moves')

def minimax_decision(state, game):
    """Given a state in a game, calculate the best move by searching
    forward all the way to the terminal states. [Figure 5.3]"""

    player = game.to_move(state)

    def max_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -infinity
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v

    def min_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = infinity
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v

    # Body of minimax_decision:
    return argmax(game.actions(state),
                  key=lambda a: min_value(game.result(state, a)))


def alphabeta_search(state, game):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = game.to_move(state)

    # Functions used by alphabeta
    def max_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -infinity
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = infinity
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alphabeta_cutoff_search:
    best_score = -infinity
    beta = infinity
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action


def alphabeta_cutoff_search(state, game, d=1, cutoff_test=None, eval_fn=None):
    """Search game to determine best action; use alpha-beta pruning.
    This version cuts off search and uses an evaluation function."""
    eval_fn = game.eval_function
    player = game.to_move(state)

    # Functions used by alphabeta
    def max_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state, player)
        v = -infinity
        for a in game.actions(state):
            st = game.result(state, a)
            #game.display(st)
            v = max(v, min_value(st, alpha, beta, depth + 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state, player)
        v = infinity
        for a in game.actions(state):
            st = game.result(state, a)
            #game.display(st)
            v = min(v, max_value(st, alpha, beta, depth + 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alphabeta_cutoff_search starts here:
    # The default test cuts off at depth d or at a terminal state
    cutoff_test = (cutoff_test or (lambda state, depth: depth > d or game.terminal_test(state)))
    eval_fn = eval_fn or (lambda state, player : game.utility(state, player))
    best_score = -infinity
    beta = infinity
    best_action = None
    for a in game.actions(state):
        st = game.result(state, a)
        #game.display(st)
        v = min_value(st, best_score, beta, 1)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

## Definição de 4 tipos de jogadores
<b>query_player</b>: oponente humano <br>
<b>random_player</b>: agente que joga de forma aleatória <br>
<b>alphabeta_player</b>: agente que joga utilizando o minimax alfabeta <br>
<b>alphabetacutoff_player</b>: agente que joga utilizando o minimax alfabeta com função de avaliação

In [ ]:
def query_player(game, state):
    """Make a move by querying standard input."""
    print("current state:")
    game.display(state)
    print("available moves: {}".format(game.actions(state)))
    print("")
    move_string = input('Your move? ')
    try:
        move = eval(move_string)
    except NameError:
        move = move_string
    return move


def random_player(game, state):
    """A player that chooses a legal move at random."""
    return random.choice(game.actions(state))

def alphabeta_player(game, state):
    return alphabeta_search(state, game)

def alphabetacutoff_player(game, state):
    return alphabeta_cutoff_search(state, game)

## Definição da classe que representa um jogo

In [ ]:
class Game:
    """A game has a utility for each
    state and a terminal test. To create a game, subclass this class and implement actions,
    result, utility, and terminal_test. You may override display and
    successors or you can inherit their default methods. You will also
    need to set the .initial attribute to the initial state; this can
    be done in the constructor."""

    def actions(self, state):
        """Return a list of the allowable moves at this point."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError

    def terminal_test(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    def to_move(self, state):
        """Return the player whose move it is in this state."""
        return state.to_move

    def display(self, state):
        """Print or otherwise display the state."""
        print(state)

    def __repr__(self):
        return '<{}>'.format(self.__class__.__name__)

    def play_game(self, *players):
        """Play an n-person, move-alternating game."""
        state = self.initial
        while True:
            print("STATE: ", state)
            for player in players:
                if state != None:
                    move = player(self, state)
                    print(" MOVE: ", move)
                    #self.display(state)
                    return self.utility(state, self.to_move(self.initial))
                else:
                    print("display")
                    self.display(state)
                    state = self.result(state, move)
                    

## JOGO: Ultimate TicTacToe

In [16]:
class UltimateTicTacToe(Game):

    ## variaveis iniciais e de configuração
    ## win_resul armazena as posições de todas as vitorias
    win_resul = [(0,4,8),(2,4,6)]
    win_resul += [(i, i+3, i+6) for i in range(3)]
    win_resul += [(i*3, i*3+1, i*3+2) for i in range(3)]
    
    def __init__(self, h=3, v=3, k=3):
        self.h = h
        self.v = v
        self.k = k
        
        ## last_move indica em qual quadrado deverá ser a proxima jogada(-1 significa qualquer lugar)
        self.last_move = -1
        ## moves contem todos os espaços livre para movimentos
        self.moves = [(i,j) for i in range(9)
                for j in range(9)]
        moves_now = self.moves.copy() ## será atualizada de acordo com a ultima jogada
        
        self.cont = 0
        
        board = [['F']*9 for _ in range(9)]
        
        self.initial = GameState(to_move='X', utility=0, board=board, moves=moves_now)

    ## recebe estado e movimento e retorna um estado com a atualização do movimento
    def result(self, state, move):
        if move not in state.moves:
            return GameState(to_move=('O' if state.to_move == 'X' else 'X'),
                             utility=self.compute_utility(state.board, move, state.to_move),
                             board=state.board, moves=state.moves)
        (x, y) = move
        board = state.board.copy()
        board[x][y] = state.to_move
        
        if(self.small_win(board[x])):
            for idx in self.moves:
                (i, j) = idx
                if (i==x):
                    print("Todos moves disponiveis após remoção do:", idx, "\n", self.moves,"\n")
                    self.moves.remove(idx)
        else:
            ##print(move)
            print("Todos moves disponiveis após remoção do:", move, "\n", self.moves,"\n")
            self.moves.remove(move)
        
        self.last_move = self.next_move(board, y)
        moves_now = self.actions(state)
        
        print("Moves disponiveis no momento desta jogada  \n", state.moves)
        print("Moves disponiveis para a proxima jogada \n", moves_now)
        self.display(state)
        
        self.cont +=1
        print("Jogada atual:", self.cont)
        return GameState(to_move=('O' if state.to_move == 'X' else 'X'),
                         utility=self.compute_utility(board, move, state.to_move),
                         board=board, moves=moves_now)

    ## imprime o tabuleiro
    def display(self, state):
        for h in range(3):
            print("-"*25)
            for i in range(3):
                line = "| "
                for j in range(3):
                    line += state.board[j+h*3][i*3] + " " + state.board[j+h*3][i*3+1] + " " + state.board[j+h*3][i*3+2] + " | "
                print(line)
        print("-"*25 +"\n")
    
    ## verifica se há vencedor num dado quadrado
    def small_win(self, square):
        for i in self.win_resul:
            (x, y, z) = i
            square_x = square[x]
            ## print(i)
            if(square_x == square[y] == square[z]) and (square_x != 'F'):
                return True
        return False

    ## verifica se há vencedor no tabuleiro
    def big_win(self, board):
        for i in self.win_resul:
            (x, y, z) = i
            small_x = self.small_win(board[x])
            if(small_x == self.small_win(board[y]) == self.small_win(board[z])) and (small_x != 'F'):
                return True
        return False

    ## verifica se o quadrado está todo preenchido
    def small_draw(self, square):
        for i in square:
            if(i=='F'):
                return False
        return True
    
    ## verifica empate no tabuleiro
    def big_draw(self, board):
        for i in board:
            if not(self.small_draw(i) or self.small_win(i)):
                return False
        return True

    ## verifica se no quadrado indicado pela ultima jogada há vitoria ou empate, caso haja last_move passa a ser -1
    def next_move(self, board, last_move):
        target_square = board[last_move]
        if(self.small_win(target_square) or self.small_draw(target_square)):
            ## print(self.small_win(target_square), self.small_draw(target_square))
            return -1
        else:
            return last_move
    
    def eval_function(self, state, player):
            v = 0
            oponent = 'O' if player == 'X' else 'X'
            
            for x in range(0, 9):
                for y in range(0, 9):
                    if state.board[x][y] == player:
                        v += 2 if (y % 2) == 1 else 3 if (not (x == 0)) else 4
                    elif state.board[x][y] == oponent:
                        v -= 2 if (y % 2) == 1 else 3 if (not (x == 0)) else 4
            u = self.utility(state, player)
            return u if not u == 0 else v

    ## redefine os movimentos legais de acordo com a ultima jogada
    def actions(self, state):
        if (self.last_move == -1):
            return self.moves
        else:
            moves_now = []
            for i in self.moves:
                x, _ = i
                if(x==self.last_move):
                    moves_now.append(i)
            return moves_now

    ## verifica se existe determinado elemento em uma lista
    def contains(self, array, el):
        for i in array:
            if (i==el):
                return True
        return False
    
    ## recebe o tabuleiro, as posições da jogada e o jogador, e retorna se a jogada é válida
    ## x é o numero do quadrado a ser jogado e y a posição nele
    def player_move(self, board, move, player):
        (x, y) = move
        if(self.contains(self.moves, move)):
            board[x][y] = player
            return True, board
        else:
            return False, board

    ## retorna o valor de utilidade do estado. 1, -1, 0 (vitoria, derrota e nenhum)
    def utility(self, state, player):
        return state.utility if player == 'X' else -state.utility
    
    ## calcula valor de utilidade
    def compute_utility(self, board, move, player):
        (x, y) = move
        true, board  = self.player_move(board, move, player)
        if(true and self.small_win(board[x])):
            if(self.big_win(board)):
                return 1000 ## utilidade para grande vitoria
            return 200 ## utilidade para pequena vitoria
        return 0 ## utilidade caso não haja vitoria
    
    ## verifica se o jogo acabou
    def terminal_test(self, state):
        return len(state.moves) == 0 or self.big_draw(state.board) or self.big_win(state.board)

In [17]:
from time import time

ttt = UltimateTicTacToe()
print("Estado inicial:")
ttt.display(ttt.initial)
s_time = time()

for _ in range(1):
    print(ttt.play_game(alphabetacutoff_player, random_player))
print(time()-s_time)

Estado inicial:
-------------------------
| F F F | F F F | F F F | 
| F F F | F F F | F F F | 
| F F F | F F F | F F F | 
-------------------------
| F F F | F F F | F F F | 
| F F F | F F F | F F F | 
| F F F | F F F | F F F | 
-------------------------
| F F F | F F F | F F F | 
| F F F | F F F | F F F | 
| F F F | F F F | F F F | 
-------------------------

STATE:  GameState(to_move='X', utility=0, board=[['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'], ['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'], ['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'], ['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'], ['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'], ['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'], ['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'], ['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'], ['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F']], moves=[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (2, 0), (2, 

In [ ]:
ttt = UltimateTicTacToe()
for _ in range(5):
    print(ttt.play_game(alphabetacutoff_player, alphabeta_player))

In [ ]:
ttt = UltimateTicTacToe()
ttt.display(ttt.initial)

my_state = GameState(
    to_move = 'X',
    utility = 0,
    board = [['F']*9 for _ in range(9)],
    moves = [(i,j) for i in range(9)
                for j in range(9)]
)

ttt.display(my_state)

In [ ]:
print("Como jogaria um jogador Random?")
ttt.display(ttt.result(my_state, random_player(ttt, my_state)))

In [ ]:
print("Como jogaria um jogador Alfabeta?")
ttt.display(ttt.result(my_state, alphabeta_player(ttt, my_state)))

In [ ]:
print("Como jogaria um jogador Alfabeta?")
ttt.display(ttt.result(my_state, alphabetacutoff_player(ttt, my_state)))

In [ ]:
from time import time
s_time = time()
for _ in range(10):
    print(ttt.play_game(random_player, alphabetacutoff_player))
print(time()-s_time)

In [ ]:
for _ in range(10):
    print(ttt.play_game(alphabeta_player, alphabetacutoff_player))

In [ ]:
s_time = time()
for _ in range(10):
    print(ttt.play_game(random_player, alphabeta_player))
print(time()-s_time)